# Импорт библиотек

In [1]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from torchvision.datasets import ImageFolder

from sklearn.metrics import classification_report, confusion_matrix, f1_score

from tqdm.notebook import tqdm, trange

import warnings

warnings.filterwarnings('ignore')

# Создание класса модели

In [2]:
class ImageClassifier(nn.Module):
    def __init__(self, num_class=2):
        super(ImageClassifier, self).__init__()

        # Сверточные слои для извлечения признаков
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        
        # Пакетная нормализация
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        
        # Функция активации ReLU
        self.relu = nn.ReLU()

        # Пулинг (уменьшение размерности)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Полносвязные слои для классификации
        self.fc1 = nn.Linear(256 * 400, 256)
        self.fc2 = nn.Linear(256, num_class) 
        
        # Регуляризация 
        self.dropout = nn.Dropout(0.5)
        

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = x.view(x.size(0), -1)  # Приравниваем размерности перед подачей в полносвязный слой
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)

        return x

# Инициализация датасета 

In [3]:
# Подготовка данных
data_transforms = transforms.Compose([
    transforms.Resize((332, 332)),  # Изменение размера изображения
    transforms.ToTensor(),  # Преобразование в тензор
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Нормализация данных
])

dataset = ImageFolder(root="D:\ProjectsData\Cats vs Dogs\PetImages", transform=data_transforms)

In [4]:
len_dataset = len(dataset)
train_size = int(len_dataset * 0.6)
val_size = int((len_dataset - train_size) * 0.5)
test_size = len_dataset - train_size - val_size

print(f"Количество наборов данных на:\nОбучение: {train_size}\nВалидацию: {val_size}\nТестирование: {test_size}")

Количество наборов данных на:
Обучение: 14998
Валидацию: 5000
Тестирование: 5000


In [5]:
# Разделение датасета на обучение, валидацию и тестирование
dataset_train, dataset_val, dataset_test = random_split(dataset, [train_size, val_size, test_size])

# Инициализация загрузчика данных

In [6]:
batch_size = 64

dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
dataloader_val = DataLoader(dataset_val)
dataloader_test = DataLoader(dataset_test)

# Инициализация модели, функции потерь, оптимизатора и контроллера скорости обучения

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

# Определение модели
model = ImageClassifier(num_class=2)
model.to(device)

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

cuda


# Обучение модели

In [ ]:
num_epochs = 10
best_accuracy = 0.0

for epoch in trange(num_epochs):
    model.train()
    train_loss = 0.0

    for images, labels in tqdm(dataloader_train):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)

    train_loss = train_loss / len(dataset_train)
    model.eval()
    valid_loss = 0.0
    y_true, y_pred = [], []
    
    with torch.no_grad():
        for images, labels in tqdm(dataloader_val):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            valid_loss += loss.item() * images.size(0)

            predictions = outputs.argmax(dim=1)
            
            y_true.extend(labels.cpu())
            y_pred.extend(predictions.cpu())

    for param_group in optimizer.param_groups:
            lr = param_group['lr']
    
    scheduler.step()
    valid_loss = valid_loss / len(dataset_val)
    print(f"Эпоха {epoch+1}/{num_epochs}\nПотери при обучении: {train_loss:.4f}\nПотери при валидации: {valid_loss:.4f}\nСкорость обучения: {lr}")
    print(classification_report(y_true, y_pred))
    accuracy = f1_score(y_true, y_pred)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), "cat_vs_dog_classifier_v1.pt")

# Тестирование модели

In [9]:
# Загрузка лучшей модели и оценка на тестовом наборе данных
model.load_state_dict(torch.load("cat_vs_dog_classifier_v1.pt"))

model.eval()
test_loss = 0.0
correct = 0
total = 0
y_true, y_pred = [], []

with torch.no_grad():
    for images, labels in tqdm(dataloader_test):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * images.size(0)

        predictions = outputs.argmax(dim=1)
        
        y_true.extend(labels.cpu())
        y_pred.extend(predictions.cpu())

test_loss = test_loss / len(dataset_test)

print(f"Потери при тестировании: {test_loss:.4f}")
print(classification_report(y_true, y_pred))
print(confusion_matrix(y_true, y_pred))

  0%|          | 0/5000 [00:00<?, ?it/s]

Потери при тестировании: 0.3526
              precision    recall  f1-score   support

           0       0.85      0.83      0.84      2474
           1       0.84      0.85      0.84      2526

    accuracy                           0.84      5000
   macro avg       0.84      0.84      0.84      5000
weighted avg       0.84      0.84      0.84      5000

[[2049  425]
 [ 372 2154]]
